In [1]:
from pymongo import MongoClient
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import *
from konlpy.utils import pprint
import numpy as np
import lda
from sklearn.pipeline import Pipeline
import pandas as pd
import statsmodels.api as sm
from collections import Counter

In [2]:
twitter = Twitter()

In [3]:
english = re.compile(r'[a-zA-Z0-9]')

In [4]:
korean = re.compile(r'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]')

# 리뷰 가져오기

In [111]:
review_dic = {}

In [8]:
rev = pd.read_table("google_review22.txt",encoding = "utf8",sep = "\t", header = None)
rev = rev.dropna()

for i in rev[0].unique():
    condition = rev[0] == i
    reviews = rev[condition][1]
    print(len(reviews))
    review_dic[int(i)] = reviews.to_list()

In [ ]:
all_reviews = []    # 리뷰 다 모으기
for i ,j in review_dic.items():
    for k in j:
        all_reviews.append(k)
#for doc in result:
#    seoul_restaurant_reviews.append(doc)

In [13]:
def do_stemming(reviews):
    result_list = []
    for i in range(len(reviews)):
        a = re.sub(' ','s',reviews[i])    #띄어쓰기 위해 s 붙힘
        b = twitter.pos(a, stem = True, norm= True)  #형태소 분석
        string = ''
        for word in b:                #문장으로 합침
            string += word[0]
        c = re.sub('s',' ',string)    #띄어쓰기 분리
        result_list.append(c)
    return result_list

In [14]:
result_list = do_stemming(all_reviews)

In [20]:
service_keyword = ['서비스','사장','알바','직원','시설','관리','교통','주차','주차','주차장','화장실','대기']

In [19]:
def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review.encode('utf-8')):
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review.encode('utf-8'))
           
            for ngram in a: 
                feature_temp.append(ngram)
            for ngram in b:
                feature_temp.append(ngram)

    return feature_temp

# ex. 서비스 감정 단어 구하기

In [96]:
#service_ngram = []
#for review in bad_result_list:
#    service_temp = get_feature_keywords(service_keyword[0:1], review)
#    if service_temp:
#        for ngram in service_temp:
#            service_ngram.append(ngram)

In [53]:
service_ngram = []
for review in result_list:
    service_temp = get_feature_keywords(service_keyword[0:1], review)
    if service_temp:
        for ngram in service_temp:
            good_service_ngram.append(ngram)

In [68]:
stop_words = [u'서비스',u'서비스도',u'서비스는',u'서비스를',u'서비스와',u'서비스로',u'서비스가',u'서비스에']

In [75]:
vect = CountVectorizer(ngram_range=(1,1), stop_words = stop_words).fit(service_ngram)
count = vect.transform(service_ngram).toarray().sum(axis=0)
result = (zip(vect.get_feature_names(), count))

In [70]:
pprint (sorted(result, key=lambda tup: tup[1], reverse=True))

[(좋다, 26),
 (하다, 17),
 (정말, 13),
 (기대하다, 12),
 (않다, 12),
 (친절하다, 10),
 (별로, 9),
 (너무, 8),
 (매우, 8),
 (없다, 8),
 (가다, 7),
 (그렇다, 7),
 (나쁘다, 7),
 (불친절, 7),
 (아니다, 7),
 (음식도, 7),
 (그냥, 6),
 (너무나, 6),
 (매장, 6),
 (별로이다, 6),
 (우리가, 6),
 (정신이, 6),
 (준대다, 6),
 (환경부터, 6),
 (괜찮다, 5),
 (맛도, 5),
 (맛이, 5),
 (면에서는, 5),
 (받다, 5),
 (별로고, 5),
 (양도, 5),
 (최악, 5),
 (그렇다고, 4),
 (들다, 4),
 (마인드나, 4),
 (만족스럽다, 4),
 (맘에, 4),
 (문제이다, 4),
 (방문하다, 4),
 (부분이, 4),
 (분이, 4),
 (살짝, 4),
 (상당하다, 4),
 (서비스나, 4),
 (서비스면에서도, 4),
 (실망스럽다, 4),
 (알바생이, 4),
 (엉망이라는, 4),
 (주시고, 4),
 (초반에, 4),
 (친절, 4),
 (컴플레인, 4),
 (테이블, 4),
 (하지만, 4),
 (ㅎㅎ, 3),
 (가격도, 3),
 (기대, 3),
 (나오다, 3),
 (다시는, 3),
 (두껍다, 3),
 (많다, 3),
 (모르다, 3),
 (무엇보다, 3),
 (문제가, 3),
 (별로라고, 3),
 (불가네요, 3),
 (생각하다, 3),
 (서비스고, 3),
 (서비스에서, 3),
 (실망, 3),
 (안좋다, 3),
 (엄청나다, 3),
 (엉망이다, 3),
 (이해, 3),
 (재방문의사, 3),
 (제대로, 3),
 (제일, 3),
 (주다, 3),
 (피자, 3),
 (회사에, 3),
 (가게, 2),
 (가격대비, 2),
 (개비추, 2),
 (개선하다필요가있다, 2),
 (겁니다, 2),
 (견디다, 2),
 (고기도, 2),
 (고기를, 2),
 (관련, 2),
 (군만두

In [71]:
vect = CountVectorizer(ngram_range=(1,1), stop_words = stop_words).fit(service_ngram)
count = vect.transform(service_ngram).toarray().sum(axis=0)
result = (zip(vect.get_feature_names(), count))

In [72]:
pprint (sorted(result, key=lambda tup: tup[1], reverse=True))

[(좋다, 245),
 (주다, 127),
 (친절하다, 64),
 (나오다, 50),
 (너무, 34),
 (정말, 33),
 (주신, 29),
 (받다, 28),
 (매우, 25),
 (않다, 22),
 (훌륭하다, 22),
 (맛도, 21),
 (기대하다, 20),
 (모두, 19),
 (분위기도, 17),
 (서비스나, 16),
 (주시고, 16),
 (나쁘다, 15),
 (그렇게, 13),
 (맛있다, 13),
 (조금, 13),
 (먹다, 12),
 (괜찮다, 11),
 (만족스럽다, 11),
 (서비스까지, 11),
 (하다, 11),
 (많이, 10),
 (아쉽다, 10),
 (아주, 10),
 (있다, 10),
 (주시는, 10),
 (넘다, 9),
 (음식, 9),
 (하나, 9),
 (나쁘다않다, 8),
 (느리다, 8),
 (마지막, 8),
 (서비스국물주니까ㅎ, 8),
 (서비스도좋다, 8),
 (서비스며, 8),
 (서비스해주다, 8),
 (아니다, 8),
 (안좋다, 8),
 (엄청, 8),
 (주신다, 8),
 (치즈볶음밥시키다, 8),
 (굉장하다, 7),
 (그냥, 7),
 (만족, 7),
 (만족하다, 7),
 (맘에, 7),
 (요리도, 7),
 (전반적으로, 7),
 (훌륭, 7),
 (ㅎㅎ, 6),
 (내부가, 6),
 (느끼다, 6),
 (다시, 6),
 (서비스의, 6),
 (신선하다, 6),
 (우동도, 6),
 (자다, 6),
 (중요하다, 6),
 (경쟁하다, 5),
 (계속, 5),
 (곳은, 5),
 (기분좋다, 5),
 (높다, 5),
 (대한, 5),
 (동시에, 5),
 (또한, 5),
 (맛에서나, 5),
 (맛은, 5),
 (별로인, 5),
 (분위기도좋다, 5),
 (서비스라, 5),
 (서비스면에서나, 5),
 (서비스임에도, 5),
 (서비스차원에서, 5),
 (스시는, 5),
 (음식맛은, 5),
 (저녁식사하다, 5),
 (제공, 5),
 (제공하다, 5),
 (존맛, 5),
 (주문과, 5

# anova 분석 실행

In [88]:
non_service_keyword = vect.get_feature_names()

In [90]:
temp_ngram = service_ngram

In [91]:
temp_ngram.extend(non_service_keyword[:376])

In [98]:
len(temp_ngram)

752

In [99]:
vect = CountVectorizer(ngram_range=(1,1), stop_words = stop_words).fit(service_ngram)
vector = vect.transform(temp_ngram).toarray()

In [100]:
voca = []
for word in vect.vocabulary_:
    voca.append(word.encode('utf-8'))

In [101]:
X = pd.DataFrame(vector, columns = voca)

In [104]:
y1 = np.ones(376)

In [105]:
y2 = np.zeros(376)

In [107]:
y = np.append(y1,y2)

In [108]:
y = pd.DataFrame(y)

In [109]:
df = pd.concat([X, y], axis=1)

In [121]:
str1 = u"""y~"""
str_list = []
for i in vect.vocabulary_.keys()[:-1]:
    str_list.append(u"""Q('{}') + """.format(i))

In [122]:
for i in str_list:
    str1 += i
str1 += u"""Q('도통')"""

In [123]:
model = sm.OLS.from_formula(str1.encode('utf-8'),data= df)

In [124]:
result = model.fit()

In [125]:
table = sm.stats.anova_lm(result)

c:\python27\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
c:\python27\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
c:\python27\lib\site-packages\scipy\stats\_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [126]:
table

,df,sum_sq,mean_sq,F,PR(>F)
Q('소박하다'),1.0,0.753004,0.753004,14.230244,1.810356e-04
Q('서비스응대엉망에'),1.0,0.000016,0.000016,0.000305,9.860777e-01
Q('부분에서는'),1.0,0.318408,0.318408,6.017272,1.450559e-02
Q('엉망이라'),1.0,0.753061,0.753061,14.231326,1.809345e-04
Q('기대하다'),1.0,0.247281,0.247281,4.673104,3.111041e-02
Q('어찌'),1.0,1.809758,1.809758,34.200738,8.966668e-09
Q('방문하다'),1.0,0.241182,0.241182,4.557858,3.325160e-02
Q('음식과'),1.0,0.241871,0.241871,4.570862,3.300246e-02
Q('알바생이'),1.0,0.485817,0.485817,9.180959,2.571555e-03
Q('친절'),1.0,0.488605,0.488605,9.233648,2.499910e-03


In [141]:
s = table['PR(>F)']
print (s<2.102125e-03).sum()
important_adj =  s[(s<2.102125e-03)].index

117


In [143]:
for i in important_adj:
    print i

Q('소박하다')
Q('엉망이라')
Q('어찌')
Q('알반지')
Q('서비스엉망')
Q('자다')
Q('형편없다')
Q('정신을')
Q('받다')
Q('양도')
Q('서비스완전개판')
Q('개비추')
Q('되다')
Q('면에서')
Q('서비스나')
Q('고기맛버리다ㅋㅋ')
Q('사람을')
Q('하다')
Q('음식점')
Q('뭔가')
Q('전체적으로')
Q('그닥')
Q('잊혀지다')
Q('대만족')
Q('레스토랑')
Q('서비스인데')
Q('엉망이라는')
Q('보나')
Q('줄은')
Q('무엇보다')
Q('음식')
Q('싫다')
Q('기대')
Q('다시')
Q('별로라는')
Q('시키다')
Q('앉다자리가')
Q('먹다')
Q('떠나다')
Q('경험하고')
Q('최악이에요')
Q('서비스라고')
Q('별루')
Q('마인드나')
Q('제주')
Q('분위기')
Q('서비스마인드')
Q('재방문하다가능성은')
Q('직원인지')
Q('완전')
Q('메뉴에')
Q('드리다')
Q('서비스교육')
Q('계산하다때')
Q('해산물')
Q('나초와')
Q('좋다')
Q('평가가')
Q('엉망이네요')
Q('다소')
Q('못한')
Q('서비스가불친절하다')
Q('서비스하다')
Q('주지만')
Q('서비스고')
Q('맘에')
Q('환경부터')
Q('실망')
Q('몇번씩')
Q('별로여서')
Q('어느')
Q('최고인데')
Q('점심메뉴가')
Q('위생때문에')
Q('아쉽다')
Q('직원들이')
Q('사업')
Q('서비스로되다')
Q('초반에')
Q('문제이다')
Q('꺼림칙한')
Q('가다')
Q('담당')
Q('그정도')
Q('실망스럽다')
Q('맛도')
Q('달다')
Q('훌륭하다')
Q('별로이다')
Q('하급')
Q('갈다때마다')
Q('때문에')
Q('하이엔드')
Q('가보다')
Q('전반적으로')
Q('부분이')
Q('매니저가')
Q('서비스면에서')
Q('카페돈까스에나')
Q('이야기를')
Q('엉성함')
Q('종업원이')
Q('재료도')
Q('좋다편인데')
Q(